In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_context("poster") 
import dateutil
import datetime
import lifelines
import statsmodels.iolib.foreign as smio
from rpy2.robjects import pandas2ri, r
import re


In [5]:
filename = "/Users/mobolajioo/Downloads/KFS8---PublicUse-101413---SPSS.sav"
w = r('foreign::read.spss("%s", to.data.frame=TRUE)' % filename)
data_df = pandas2ri.ri2py(w)
data_df.head()


,MPRID,credrisk_0,credrisk_1,credrisk_2,credrisk_3,credrisk_4,credrisk_5,credrisk_6,credrisk_7,Final_Status_Code_0,B1_Bus_start_0,B2a_Legal_Status_0,B2b_Prev_Legal_Status_0,B3_EIN_0,B4_EIN_Year_0,B5_Schedule_C_0,B6_Schedule_C_Year_0,B7_UI_0,B8_UI_Year_0,B10_FICA_0,B11_FICA_Year_0,C2_Owners_0,C3a_Owner_Operators_0,C4_NumOwners_Confirm_0,C5_Num_Employees_0,C6_Num_FT_Employees_0,C7_Num_PT_Employees_0,C8_Primary_Loc_0,D1a_Provide_Service_0,D1b_Provide_Product_0,D2_Comp_Advantage_0,D4_a_Lic_Out_Patent_0,D4_b_Lic_Out_Copyright_0,D4_c_Lic_Out_Trademark_0,D5_a_Lic_In_Patent_0,D5_b_Lic_In_Copyright_0,D5_c_Lic_In_Trademark_0,D6_Have_Sales_0,D7_Perc_Sales_Bus_0,D7_Perc_Sales_Govt_0,D7_Perc_Sales_Indiv_0,E1_a_Num_Human_Res_0,E1_b_Num_Sales_0,E1_c_Num_Exec_Admin_0,E1_d_Num_ResDev_0,E1_e_Num_Prod_Manu_0,E1_f_Num_Gen_Admin_0,E1_g_Num_Fin_Admin_0,E1_h_Num_Other_0,E2a_FT_Emp_Bonus_Plan_0,...,Age_Owner_02_R_7,Age_Owner_03_R_7,Age_Owner_04_R_7,Age_Owner_05_R_7,Age_Owner_06_R_7,Age_Owner_07_R_7,Age_Owner_08_R_7,Age_Owner_09_R_7,Age_Owner_10_R_7,Age_Owner_11_R_7,Age_Owner_12_R_7,Age_Owner_13_R_7,Age_Owner_14_R_7,Age_Owner_15_R_7,Total_Hours_Owner_01_R_7,Total_Hours_Owner_02_R_7,Total_Hours_Owner_03_R_7,Total_Hours_Owner_04_R_7,Total_Hours_Owner_05_R_7,Total_Hours_Owner_06_R_7,Total_Hours_Owner_07_R_7,Total_Hours_Owner_08_R_7,Total_Hours_Owner_09_R_7,Total_Hours_Owner_10_R_7,Total_Hours_Owner_11_R_7,Total_Hours_Owner_12_R_7,Total_Hours_Owner_13_R_7,Total_Hours_Owner_14_R_7,Total_Hours_Owner_15_R_7,Total_Patents_7,Total_Copyrights_7,Total_Trademarks_7,wgt_final_f7_long_7,wgt_final_f7_7,Email_0,Website_0,Email_1,Website_1,Email_2,Website_2,Email_3,Website_3,Email_4,Website_4,Email_5,Website_5,Email_6,Website_6,Email_7,Website_7
1,10000016,2,2,2,2,3,NaN,NaN,,10,3,4,0,1,2,NaN,NaN,1,1,1,1,1,1,1,04,04,00,2,1,1,1,NaN,1,0,0,1,0,1,90,10,0,00,01,01,02,02,01,01,00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,3.098758,1.949016,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000090,3,3,3,3,2,3,3,3,30,3,1,0,0,NaN,1,1,0,NaN,0,NaN,1,1,1,01,01,00,1,1,0,1,NaN,NaN,NaN,0,0,0,1,1,0,99,00,01,01,01,00,01,01,01,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00,000,00,25.186329,19.858225,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,10000320,5,5,4,4,2,2,2,,10,3,4,0,1,2,NaN,NaN,1,1,1,1,1,1,1,02,00,02,2,1,0,0,NaN,NaN,NaN,0,0,0,1,0,0,100,02,02,01,01,01,02,01,00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000375,3,3,3,3,2,2,3,2,30,4,1,0,0,NaN,1,1,0,NaN,0,NaN,1,1,1,01,01,00,2,1,1,0,NaN,NaN,NaN,0,0,0,1,10,5,85,01,01,01,01,01,01,01,00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,0.000000,0.000000,1,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10000391,3,3,3,3,3,NaN,NaN,,10,3,2,0,1,2,NaN,NaN,0,NaN,0,NaN,1,1,1,00,,,1,1,0,1,NaN,NaN,NaN,0,0,0,1,100,0,0,,,,,,,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,8.429062,8.271661,1,0,1,0,1,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data_arr = np.asarray(data_df)

In [7]:
def read_year_data(year,df=data_df):
    filter_col = [col for col in list(df) if col.endswith(year)]
    df = df[['MPRID']+filter_col].sort_index(axis=1)
    return df

In [8]:
data_df_02 = read_year_data('_2')
data_df_02.head()

,A10_Out_Of_Business_2,A11a_Bankruptcy_2,A5New_Actively_involved_2,A5_Actively_involved_2,A8_Addr_ver_2,Age_Owner_01_R_2,Age_Owner_02_R_2,Age_Owner_03_R_2,Age_Owner_04_R_2,Age_Owner_05_R_2,Age_Owner_06_R_2,Age_Owner_07_R_2,Age_Owner_08_R_2,Age_Owner_09_R_2,Age_Owner_10_R_2,C10_MoreLocations_2,C11_Num_Locations_2,C11a_2ndOpening_Month_2,C11a_2ndOpening_Year_2,C1a_NAICS_verification_2,C1z2_Legal_Status_2,C1z_Confirm_Legal_Status_2,C2_Owners_2,C3a_Owner_Operators_2,C4_NumOwners_Confirm_2,C5_Num_Employees_2,C5b_Num_Employees_Change_2,C5c_Num_Employees_Change_Amt_2,C6_Num_FT_Employees_2,C7_Num_PT_Employees_2,C8_Primary_Loc_2,C8z_Primary_Loc_Confirm_2,C9_Loc_Change_Reason_2,Census_Region_2,D1a_Provide_Service_2,D1b_Provide_Product_2,D2_Comp_Advantage_2,D4_a_Lic_Out_Patent_2,D4_b_Lic_Out_Copyright_2,D4_c_Lic_Out_Trademark_2,D5_a_Lic_In_Patent_2,D5_b_Lic_In_Copyright_2,D5_c_Lic_In_Trademark_2,D6_Have_Sales_2,D7_Perc_Sales_Bus_2,D7_Perc_Sales_Govt_2,D7_Perc_Sales_Indiv_2,E1_a_Num_Human_Res_2,E1_b_Num_Sales_2,E1_c_Num_Exec_Admin_2,...,Tot_PersLoan_Bank_Owed_Othrs_R_2,Tot_PersLoan_Fam_OthrOwners_R_2,Tot_PersLoan_Fam_Owed_Othrs_R_2,Tot_PersLoan_Other_Owed_Resp_R_2,Tot_PersLoan_Othr_Owed_Othrs_R_2,Tot_Pers_CredCard_Bal_Others_R_2,Tot_Pers_CredCard_Bal_Resp_R_2,Tot_Pers_CredCard_Line_Othrs_R_2,Tot_Pers_CredCard_Line_Resp_R_2,Tot_Pers_Debt_Other_Owners_R_2,Tot_Pers_Debt_Owed_OthrOwnrs_R_2,Tot_Pers_Debt_Owed_Resp_R_2,Tot_Pers_Debt_Resp_R_2,Tot_Pers_Loan_Bank_Others_R_2,Tot_Pers_Loan_Bank_Owed_Resp_R_2,Tot_Pers_Loan_Bank_Resp_R_2,Tot_Pers_Loan_Fam_Owed_Resp_R_2,Tot_Pers_Loan_Fam_Resp_R_2,Tot_Pers_Loan_Other_Owners_R_2,Tot_Pers_Loan_Other_Resp_R_2,Tot_Pers_Other_Other_Owners_R_2,Tot_Pers_Other_Owed_Others_R_2,Tot_Pers_Other_Owed_Resp_R_2,Tot_Pers_Other_Resp_R_2,Tot_Personal_Use_2006_R_2,Tot_Profit_R_2,Tot_Revenue_R_2,Tot_Trade_Finan_R_2,Tot_Wages_R_2,TotalOwners_2,Total_Copyrights_2,Total_Hours_Owner_01_R_2,Total_Hours_Owner_02_R_2,Total_Hours_Owner_03_R_2,Total_Hours_Owner_04_R_2,Total_Hours_Owner_05_R_2,Total_Hours_Owner_06_R_2,Total_Hours_Owner_07_R_2,Total_Hours_Owner_08_R_2,Total_Hours_Owner_09_R_2,Total_Hours_Owner_10_R_2,Total_Patents_2,Total_Trademarks_2,Website_2,Wgt_Final_F2_2,credrisk_2,hightech_2,techempl_2,techgenr_2,wgt_final_f12_long_2
1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.204142,2,1,1,1,3.200374
2,NaN,NaN,NaN,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1,1,1,1,1,1,01,3,NaN,01,00,1,1,NaN,1,1,0,0,NaN,NaN,NaN,0,0,0,1,0,0,100,00,01,01,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,5,NaN,NaN,3,3,NaN,0,0,0,0,NaN,0,NaN,NaN,0,0,2,3,0,0,0,1,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,20.625456,3,0,0,0,20.713455
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,4,0,0,0,0.000000
4,NaN,NaN,NaN,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1,1,1,1,1,1,00,3,NaN,,,2,1,NaN,3,1,1,0,NaN,NaN,0,0,0,0,1,0,10,90,,,,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN,0,NaN,NaN,0,0,0,4,7,6,NaN,1,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,19.340121,3,0,0,0,19.430043
5,NaN,NaN,NaN,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1,2,1,1,1,1,00,3,NaN,,,1,1,NaN,1,1,0,1,NaN,NaN,NaN,0,0,0,1,100,0,0,,,,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0,0,NaN,0,NaN,NaN,0,0,2,0,0,0,NaN,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,7.686683,3,0,0,0,7.753953


In [9]:
#read file containing relevant column names line by line
fname = '/Users/mobolajioo/Desktop/Software Engineeting & Data Science Resources/column-names'
with open(fname) as f:
    rel_col_names = f.readlines() #names of relevant columns based on reading Kauffman pdf
rel_col_names = [x.rstrip() for x in rel_col_names] #remove whitespace characters like `\n` at the end of each line
#rel_col_names

In [65]:
#filter relevant columns from intial columns
rel_col_from_all_cols = [i for i in data_df_02.columns for j in rel_col_names if i.lower().startswith(j.lower())]
data_df_02_rel_col = data_df_02[['MPRID']+list(set(rel_col_from_all_cols))]
data_df_02_rel_col.head()
data_df_02_rel_col.shape #(4928, 86)

(4928, 71)

In [66]:
data_df_02_rel_col.head()

,MPRID,Tot_Liab_R_2,Tot_Equity_Spouse_AllYrs_R_2,Tot_Equity_Owner_10_R_2,D5_b_Lic_In_Copyright_2,Tot_Liab_Pension_R_2,Tot_Equity_Owner_2_R_2,C2_Owners_2,Tot_Equity_AllYrs_Owner_07_R_2,Tot_Equity_Parents_AllYrs_R_2,D1b_Provide_Product_2,D7_Perc_Sales_Bus_2,D7_Perc_Sales_Govt_2,Tot_Equity_NonOwnerOperators_R_2,D4_b_Lic_Out_Copyright_2,Tot_Equity_Parents_R_2,Tot_Equity_Owner_6_R_2,Tot_Liab_Other_R_2,Tot_Equity_Govt_AllYrs_R_2,C3a_Owner_Operators_2,Tot_Equity_AllYrs_Owner_05_R_2,Tot_Equity_Owner_9_R_2,Tot_Equity_AllYrs_Owner_09_R_2,Tot_Equity_Owner_Operators_R_2,D2_Comp_Advantage_2,A10_Out_Of_Business_2,Tot_Equity_Companies_R_2,Tot_Equity_Owner_3_R_2,Tot_Equity_OwnerOper_AllYrs_R_2,Tot_Debt_Owner_Operators_R_2,Tot_Equity_Other_R_2,D1a_Provide_Service_2,Tot_Equity_Govt_R_2,TotalOwners_2,Tot_Assets_R_2,Tot_Equity_Owner_4_R_2,Tot_Equity_Vent_Cap_R_2,Tot_Debt_R_2,D5_a_Lic_In_Patent_2,Tot_Bus_Debt_Owed_R_2,Tot_Equity_AllYrs_Owner_02_R_2,Tot_Equity_Comp_AllYrs_R_2,Tot_Pers_Debt_Owed_OthrOwnrs_R_2,Tot_Equity_Owner_1_R_2,Tot_Equity_Vent_Cap_AllYrs_R_2,Tot_Equity_AllYrs_Owner_10_R_2,D5_c_Lic_In_Trademark_2,Tot_Equity_Spouse_R_2,D7_Perc_Sales_Indiv_2,Tot_Equity_R_2,Tot_Debt_Bus_R_2,Tot_Pers_Debt_Resp_R_2,Tot_Equity_Angels_R_2,Tot_Equity_NonOwnerOp_AllYrs_R_2,Tot_Equity_AllYrs_Owner_08_R_2,Tot_Equity_Angels_AllYrs_R_2,Tot_Liab_Acct_Pay_R_2,C1a_NAICS_verification_2,Tot_Pers_Debt_Other_Owners_R_2,Tot_Equity_AllYrs_Owner_01_R_2,Tot_Equity_Owner_5_R_2,D4_c_Lic_Out_Trademark_2,Tot_Equity_Other_AllYrs_R_2,D6_Have_Sales_2,Tot_Equity_AllYrs_Owner_03_R_2,Tot_Equity_Owner_8_R_2,Tot_Equity_AllYrs_Owner_06_R_2,Tot_Equity_AllYrs_Owner_04_R_2,Tot_Pers_Debt_Owed_Resp_R_2,Tot_Equity_Owner_7_R_2,D4_a_Lic_Out_Patent_2
1,10000016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000090,0,NaN,NaN,0,0,NaN,1,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,0,NaN,1,NaN,NaN,NaN,0,0,NaN,NaN,NaN,6,3,NaN,1,NaN,1,0,NaN,NaN,3,0,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,100,0,0,3,NaN,NaN,NaN,NaN,0,1,NaN,6,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,3,NaN,NaN
3,10000320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000375,0,NaN,NaN,0,0,NaN,1,NaN,NaN,1,0,10,NaN,NaN,NaN,NaN,0,NaN,1,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1,0,NaN,1,NaN,1,6,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,90,0,0,0,NaN,NaN,NaN,NaN,0,1,NaN,0,NaN,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10000391,0,NaN,NaN,0,0,NaN,1,NaN,NaN,0,100,0,0,NaN,0,NaN,0,NaN,1,NaN,NaN,NaN,3,1,NaN,0,NaN,5,0,0,1,0,1,7,NaN,0,0,0,NaN,NaN,NaN,NaN,3,NaN,NaN,0,0,0,3,0,0,0,NaN,NaN,NaN,0,1,NaN,5,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
#get list of columns containing 'Owner_'
owner_cols = [k for k in list(set(data_df_02_rel_col.columns)) if re.search(r'Owner_\d',k)]
len(owner_cols)

20

In [68]:
#get list of columns starting with 'Tot_Equity_AllYrs_Owner_' and 'Tot_Equity_Owner_'
cols_Tot_Equity_AllYrs_Owner_ = [k for k in list(set(data_df_02_rel_col.columns)) if re.search
                                (r'^Tot_Equity_AllYrs_Owner_\d',k)]

cols_Tot_Equity_Owner_ = [k for k in list(set(data_df_02_rel_col.columns)) if re.search
                          (r'^Tot_Equity_Owner_\d',k)]

In [69]:
# teao is acronym for total equity allyrs owner
teao_df = data_df_02_rel_col[['MPRID']+cols_Tot_Equity_AllYrs_Owner_]

# teo is acronym for total equity owner
teo_df = data_df_02_rel_col[['MPRID']+cols_Tot_Equity_Owner_]

In [70]:
#create dataframe with means of teao & teo per business (across 10 owners) 
#and index by business id 
mean_teao_df = teao_df[cols_Tot_Equity_AllYrs_Owner_].mean(axis=1, skipna=True)
mean_teo_df = teo_df[cols_Tot_Equity_Owner_].mean(axis=1, skipna=True)

mean_teao_df.index = teao_df['MPRID'].values
mean_teo_df.index = teo_df['MPRID'].values

mean_teao_df = pd.DataFrame(mean_teao_df)
mean_teo_df = pd.DataFrame(mean_teo_df)

mean_teao_df = mean_teao_df.fillna(0)
mean_teo_df = mean_teo_df.fillna(0)

mean_teao_df = mean_teao_df.reset_index()
mean_teo_df = mean_teo_df.reset_index()

mean_teao_df.columns = ['MPRID','Mean_TEAO']
mean_teo_df.columns = ['MPRID','Mean_TEO']



In [71]:
#merge dataframe with means of teao (mean_teao_df) and means of teo (mean_teo_df)
#into year_2 dataframe (data_df_02_rel_col) 

data_df_02_rel_col = pd.merge(mean_teao_df, data_df_02_rel_col, on='MPRID')
data_df_02_rel_col = pd.merge(mean_teo_df, data_df_02_rel_col, on='MPRID')


In [72]:
#drop columns with teao and teo data from year_2 dataframe since you now have teao and teo means
data_df_02_teo_and_teao = data_df_02_rel_col.drop(cols_Tot_Equity_AllYrs_Owner_
                                                       +cols_Tot_Equity_Owner_, axis=1)

In [73]:
data_df_02_teo_and_teao.head()

,MPRID,Mean_TEO,Mean_TEAO,Tot_Liab_R_2,Tot_Equity_Spouse_AllYrs_R_2,D5_b_Lic_In_Copyright_2,Tot_Liab_Pension_R_2,C2_Owners_2,Tot_Equity_Parents_AllYrs_R_2,D1b_Provide_Product_2,D7_Perc_Sales_Bus_2,D7_Perc_Sales_Govt_2,Tot_Equity_NonOwnerOperators_R_2,D4_b_Lic_Out_Copyright_2,Tot_Equity_Parents_R_2,Tot_Liab_Other_R_2,Tot_Equity_Govt_AllYrs_R_2,C3a_Owner_Operators_2,Tot_Equity_Owner_Operators_R_2,D2_Comp_Advantage_2,A10_Out_Of_Business_2,Tot_Equity_Companies_R_2,Tot_Equity_OwnerOper_AllYrs_R_2,Tot_Debt_Owner_Operators_R_2,Tot_Equity_Other_R_2,D1a_Provide_Service_2,Tot_Equity_Govt_R_2,TotalOwners_2,Tot_Assets_R_2,Tot_Equity_Vent_Cap_R_2,Tot_Debt_R_2,D5_a_Lic_In_Patent_2,Tot_Bus_Debt_Owed_R_2,Tot_Equity_Comp_AllYrs_R_2,Tot_Pers_Debt_Owed_OthrOwnrs_R_2,Tot_Equity_Vent_Cap_AllYrs_R_2,D5_c_Lic_In_Trademark_2,Tot_Equity_Spouse_R_2,D7_Perc_Sales_Indiv_2,Tot_Equity_R_2,Tot_Debt_Bus_R_2,Tot_Pers_Debt_Resp_R_2,Tot_Equity_Angels_R_2,Tot_Equity_NonOwnerOp_AllYrs_R_2,Tot_Equity_Angels_AllYrs_R_2,Tot_Liab_Acct_Pay_R_2,C1a_NAICS_verification_2,Tot_Pers_Debt_Other_Owners_R_2,D4_c_Lic_Out_Trademark_2,Tot_Equity_Other_AllYrs_R_2,D6_Have_Sales_2,Tot_Pers_Debt_Owed_Resp_R_2,D4_a_Lic_Out_Patent_2
0,10000016,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000090,0,6,0,NaN,0,0,1,NaN,0,0,0,NaN,NaN,NaN,0,NaN,1,0,0,NaN,NaN,6,3,NaN,1,NaN,1,0,NaN,3,0,NaN,NaN,NaN,NaN,0,NaN,100,0,0,3,NaN,NaN,NaN,0,1,NaN,NaN,NaN,1,3,NaN
2,10000320,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000375,0,0,0,NaN,0,0,1,NaN,1,0,10,NaN,NaN,NaN,0,NaN,1,0,0,NaN,NaN,1,0,NaN,1,NaN,1,6,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,90,0,0,0,NaN,NaN,NaN,0,1,NaN,0,NaN,1,NaN,NaN
4,10000391,3,5,0,NaN,0,0,1,NaN,0,100,0,0,NaN,0,0,NaN,1,3,1,NaN,0,5,0,0,1,0,1,7,0,0,0,NaN,NaN,NaN,NaN,0,0,0,3,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,1,NaN,NaN


In [74]:
data_df_02_teo_and_teao.shape

(4928, 53)

In [62]:
#list(pd.DataFrame(er).sort_index().T.columns)
#data_df_02_rel_col[owner_cols].shape

In [63]:
#er = (data_df_02_rel_col[owner_cols].isnull().sum()/4928)*100
#er = er.astype(int)
#er